# MQ-135

## Specifications

### Standard Work Condition

    Vc (Circuit Voltage)       :  5v +/- 0.1
    Vh (Heating Voltage)       :  5v +/- 0.1
    Rl (Load Resistance)       :  can adjust
    Rh (Heater Resistance)     :  33 Ohm +/- 5%
    Ph (Heating consumption)   :  less than 800mw
    

### Sensitivity Characteristic:

    Rs (Sensing Resistance)    : 30kOhm - 200kOhm (100ppm NH3)
    Preheat Time               : Over 24 hour
    
    Detecting Concentration Scope:
        NH3     :  10ppm - 300ppm 
        Benzene :  10ppm - 1000ppm
        Alcohol :  10ppm - 300ppm


In [1]:
#%pip install numpy

import numpy as np
import math

In [2]:
## Gas concentration values referenced in the datasheet

# Carbon Dioxide
co2 = {
    'p1' : {'x': 10,   'y': 2.298},
    'p2' : {'x': 100,  'y': 1.053},
    'p3' : {'x': 200,  'y': 0.803}
}

# Carbon Monoxide
co = {
    'p1' : {'x': 10,   'y': 2.837},
    'p2' : {'x': 100,  'y': 1.539},
    'p3' : {'x': 200,  'y': 1.349}
}

alcohol = {
    'p1' : {'x': 10,   'y': 1.894},
    'p2' : {'x': 100,  'y': 0.933},
    'p3' : {'x': 200,  'y': 0.734}
}

# Ammonium
nh4 = {
    'p1' : {'x': 10,   'y': 2.545},
    'p2' : {'x': 100,  'y': 1.000},
    'p3' : {'x': 200,  'y': 0.770}
}

tolueno = {
    'p1' : {'x': 10,   'y': 1.543},
    'p2' : {'x': 100,  'y': 0.803},
    'p3' : {'x': 200,  'y': 0.643}
}

acetona = {
    'p1' : {'x': 10,   'y': 1.419},
    'p2' : {'x': 100,  'y': 0.742},
    'p3' : {'x': 200,  'y': 0.587}
}


AIR = 3.603

In [3]:
# RS value calculation
Rs    = lambda Vcc, Vrl, RL : ((Vcc * RL) / Vrl) - RL

# R0 value calculation
R0    = lambda Rs, air : Rs / air

# RS/Ro value calculation (ratio)
Ratio = lambda Rs, R0: Rs / R0

# log(y) = m.log(x) + b
# m
M     = lambda p1, p2 : (math.log10( p2['y'] / p1['y'] )) / (math.log10(p2['x']/p1['x']))
# b
B     = lambda p, m : math.log10(p['y']) - (m * math.log10(p['x']))

# PPM value calculation 
PPM   = lambda  y, m, b: 10 ** ((math.log10(y) - b) / m)

In [4]:
## Calculate the points to be used for the calculation of _m_ and _b_ according to the _ratio_ value 
def getPoints(racio, values):
    size = len(values)
    for idx in range(size - 1):
        if racio >= values[idx]['y']:
            if idx == 0:
                return (values[0], values[1])
            return(values[idx - 1], values[idx])
    return(values[size - 2], values[size - 1])

In [5]:
## Test the _getPoints_ function
"""
points = ['p1', 'p2', 'p3']
_nh4  = [ nh4[n]      for n in list(nh4)      if n in points]


racio  = 1
p1, p2  = getPoints(racio, _nh4)

m = M(p1,p2)
b = B(p1, m)

ppm  = PPM(racio, m, b)
print(f"{p1}\n{p2}")
print(f"[ m: {m:.2f} || b: {b:.2f} ]")
print(f"{ppm:.0f}ppm")
"""
print()

In [7]:
points = ['p1', 'p2', 'p3']

_co2     = [ co2[n]     for n in list(co2)     if n in points]
_co      = [ co[n]      for n in list(co)      if n in points]
_alcohol = [ alcohol[n] for n in list(alcohol) if n in points]
_nh4     = [ nh4[n]     for n in list(nh4)     if n in points]
_tolueno = [ tolueno[n] for n in list(tolueno) if n in points]
_acetona = [ acetona[n] for n in list(acetona) if n in points]

#print(_co2)
#print(_co)
#print(_alcohol)
#print(_nh4)
#print(_tolueno)
#print(_acetona)

[{'x': 10, 'y': 2.298}, {'x': 100, 'y': 1.053}, {'x': 200, 'y': 0.803}]
[{'x': 10, 'y': 2.837}, {'x': 100, 'y': 1.539}, {'x': 200, 'y': 1.349}]
[{'x': 10, 'y': 1.894}, {'x': 100, 'y': 0.933}, {'x': 200, 'y': 0.734}]
[{'x': 10, 'y': 2.545}, {'x': 100, 'y': 1.0}, {'x': 200, 'y': 0.77}]
[{'x': 10, 'y': 1.543}, {'x': 100, 'y': 0.803}, {'x': 200, 'y': 0.643}]
[{'x': 10, 'y': 1.419}, {'x': 100, 'y': 0.742}, {'x': 200, 'y': 0.587}]


In [9]:
_ratios   = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]

co2_ppm     = []
co_ppm      = []
alcohol_ppm = []
nh4_ppm     = []
tolueno_ppm = []
acetona_ppm = []

for r in _ratios:
    p1, p2  = getPoints(r, _co2)
    m = M(p1,p2)
    b = B(p1, m)
    co2_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _co)
    m = M(p1,p2)
    b = B(p1, m)
    co_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _alcohol)
    m = M(p1,p2)
    b = B(p1, m)
    alcohol_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _nh4)
    m = M(p1,p2)
    b = B(p1, m)
    nh4_ppm.append(PPM(r, m, b))
    
    p1, p2  = getPoints(r, _tolueno)
    m = M(p1,p2)
    b = B(p1, m)
    tolueno_ppm.append(PPM(r, m, b))
        
    p1, p2  = getPoints(r, _acetona)
    m = M(p1,p2)
    b = B(p1, m)
    acetona_ppm.append(PPM(r, m, b))
    
    
print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Ratio','CO2', 'CO', 'Alcohol', 'NH4', 'Tolueno', 'Acetona'))
for i in range(len(_ratios)):
    print ("{:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
           .format( _ratios[i],
                   round(co2_ppm[i],3),
                   round(co_ppm[i],3),
                   round(alcohol_ppm[i],3),
                   round(nh4_ppm[i],3),
                   round(tolueno_ppm[i],3),
                   round(acetona_ppm[i],3)
                  ))

Ratio    CO2           CO            Alcohol       NH4           Tolueno       Acetona      
10       0.131         0.087         0.045         0.343         0.014         0.01         
9        0.178         0.13          0.063         0.444         0.02          0.014        
8        0.252         0.202         0.092         0.594         0.03          0.022        
7        0.374         0.334         0.142         0.826         0.048         0.035        
6        0.589         0.596         0.235         1.208         0.083         0.06         
5        1.009         1.184         0.426         1.893         0.158         0.114        
4        1.949         2.743         0.879         3.281         0.348         0.252        
3        4.554         8.103         2.241         6.667         0.959         0.7          
2        15.065        37.291        8.377         18.112        4.007         2.956        
1        114.119       965.902       79.809        100.0         46.14

In [10]:
# MQ 135 - sensor 1
sensor1 = {}

sensor1['Vcc'] = 5      #volts
sensor1['Vrl'] = 0.28   #volts
sensor1['RL']  = 10     #kOhms
sensor1['AIR'] = AIR   

sensor1['rs'] = Rs(sensor1['Vcc'], sensor1['Vrl'], sensor1['RL'])
sensor1['ro'] = R0(sensor1['rs'], sensor1['AIR'])
sensor1['ratio'] = Ratio(sensor1['rs'], sensor1['ro'])

# MQ 135 - sensor 2
sensor2 = {}

sensor2['Vcc'] = 5      #volts
sensor2['Vrl'] = 0.48   #volts
sensor2['RL']  = 10     #kOhms
sensor2['AIR'] = AIR   

sensor2['rs'] = Rs(sensor2['Vcc'], sensor2['Vrl'], sensor2['RL'])
sensor2['ro'] = R0(sensor2['rs'], sensor2['AIR'])
sensor2['ratio'] = Ratio(sensor2['rs'], sensor2['ro'])



print ("{:<8} {:<8} {:<8} {:<8}".format('Sensor','Rs', 'Ro','Ratio'))
print ("{:<8} {:<8} {:<8} {:<8}".format(1, round(sensor1['rs'],2), round(sensor1['ro'], 2), round(sensor1['ratio'], 2)))
print ("{:<8} {:<8} {:<8} {:<8}".format(2, round(sensor2['rs'],2), round(sensor2['ro'], 2), round(sensor2['ratio'], 2)))
    

Sensor   Rs       Ro       Ratio   
1        168.57   46.79    3.6     
2        94.17    26.14    3.6     


In [15]:
## PPM
sensor1['co2'] = {}
sensor1['co2']['p1'], sensor1['co2']['p2'] = getPoints(sensor1['ratio'], _co2)
sensor1['co2']['m'] = M(sensor1['co2']['p1'], sensor1['co2']['p2'])
sensor1['co2']['b'] = B(sensor1['co2']['p1'], sensor1['co2']['m'])

sensor1['co'] = {}
sensor1['co']['p1'], sensor1['co']['p2'] = getPoints(sensor1['ratio'], _co)
sensor1['co']['m'] = M(sensor1['co']['p1'], sensor1['co']['p2'])
sensor1['co']['b'] = B(sensor1['co']['p1'], sensor1['co']['m'])

sensor1['alcohol'] = {}
sensor1['alcohol']['p1'], sensor1['alcohol']['p2'] = getPoints(sensor1['ratio'], _alcohol)
sensor1['alcohol']['m'] = M(sensor1['alcohol']['p1'], sensor1['alcohol']['p2'])
sensor1['alcohol']['b'] = B(sensor1['alcohol']['p1'], sensor1['alcohol']['m'])

sensor1['nh4'] = {}
sensor1['nh4']['p1'], sensor1['nh4']['p2'] = getPoints(sensor1['ratio'], _nh4)
sensor1['nh4']['m'] = M(sensor1['nh4']['p1'], sensor1['nh4']['p2'])
sensor1['nh4']['b'] = B(sensor1['nh4']['p1'], sensor1['nh4']['m'])


sensor1['tolueno'] = {}
sensor1['tolueno']['p1'], sensor1['tolueno']['p2'] = getPoints(sensor1['ratio'], _tolueno)
sensor1['tolueno']['m'] = M(sensor1['tolueno']['p1'], sensor1['tolueno']['p2'])
sensor1['tolueno']['b'] = B(sensor1['tolueno']['p1'], sensor1['tolueno']['m'])

sensor1['acetona'] = {}
sensor1['acetona']['p1'], sensor1['acetona']['p2'] = getPoints(sensor1['ratio'], _acetona)
sensor1['acetona']['m'] = M(sensor1['acetona']['p1'], sensor1['acetona']['p2'])
sensor1['acetona']['b'] = B(sensor1['acetona']['p1'], sensor1['acetona']['m'])

####

sensor2['co2'] = {}
sensor2['co2']['p1'], sensor2['co2']['p2'] = getPoints(sensor2['ratio'], _co2)
sensor2['co2']['m'] = M(sensor2['co2']['p1'], sensor2['co2']['p2'])
sensor2['co2']['b'] = B(sensor2['co2']['p1'], sensor2['co2']['m'])

sensor2['co'] = {}
sensor2['co']['p1'], sensor2['co']['p2'] = getPoints(sensor2['ratio'], _co)
sensor2['co']['m'] = M(sensor2['co']['p1'], sensor2['co']['p2'])
sensor2['co']['b'] = B(sensor2['co']['p1'], sensor2['co']['m'])

sensor2['alcohol'] = {}
sensor2['alcohol']['p1'], sensor2['alcohol']['p2'] = getPoints(sensor2['ratio'], _alcohol)
sensor2['alcohol']['m'] = M(sensor2['alcohol']['p1'], sensor2['alcohol']['p2'])
sensor2['alcohol']['b'] = B(sensor2['alcohol']['p1'], sensor2['alcohol']['m'])

sensor2['nh4'] = {}
sensor2['nh4']['p1'], sensor2['nh4']['p2'] = getPoints(sensor2['ratio'], _nh4)
sensor2['nh4']['m'] = M(sensor2['nh4']['p1'], sensor2['nh4']['p2'])
sensor2['nh4']['b'] = B(sensor2['nh4']['p1'], sensor2['nh4']['m'])

sensor2['tolueno'] = {}
sensor2['tolueno']['p1'], sensor2['tolueno']['p2'] = getPoints(sensor2['ratio'], _tolueno)
sensor2['tolueno']['m'] = M(sensor2['tolueno']['p1'], sensor2['tolueno']['p2'])
sensor2['tolueno']['b'] = B(sensor2['tolueno']['p1'], sensor2['tolueno']['m'])


sensor2['acetona'] = {}
sensor2['acetona']['p1'], sensor2['acetona']['p2'] = getPoints(sensor2['ratio'], _acetona)
sensor2['acetona']['m'] = M(sensor2['acetona']['p1'], sensor2['acetona']['p2'])
sensor2['acetona']['b'] = B(sensor2['acetona']['p1'], sensor2['acetona']['m'])


####

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('Volt', 'Ratio','CO2', 'CO', 'Alcohol', 'NH4', 'Tolueno', 'Acetona'))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor1['Vrl'], 3),
           round(sensor1['ratio'], 2),
           round(PPM(sensor1['ratio'], sensor1['co2']['m'],     sensor1['co2']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['co']['m'],      sensor1['co']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['alcohol']['m'], sensor1['alcohol']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['nh4']['m'],     sensor1['nh4']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['tolueno']['m'], sensor1['tolueno']['b']),2),
           round(PPM(sensor1['ratio'], sensor1['acetona']['m'], sensor1['acetona']['b']),2),
          ))

print ("{:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
   .format(round(sensor2['Vrl'], 3),
           round(sensor2['ratio'], 2),
           round(PPM(sensor2['ratio'], sensor2['co2']['m'],     sensor2['co2']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['co']['m'],      sensor2['co']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['alcohol']['m'], sensor2['alcohol']['b']),2),           
           round(PPM(sensor2['ratio'], sensor2['nh4']['m'],     sensor2['nh4']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['tolueno']['m'], sensor2['tolueno']['b']),2),
           round(PPM(sensor2['ratio'], sensor2['acetona']['m'], sensor2['acetona']['b']),2),
          ))

Volt     Ratio    CO2           CO            Alcohol       NH4           Tolueno       Acetona      
0.28     3.6      2.65          4.07          1.24          4.24          0.5           0.37         
0.48     3.6      2.65          4.07          1.24          4.24          0.5           0.37         


In [22]:
## Projection - Increasing VRL

# Sensor 1

Vcc    = 5 #volts
vrl    = np.arange(sensor1['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor1['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('','Volt', 'Ratio','CO2', 'CO', 'Alcohol', 'NH4', 'Tolueno', 'Acetona'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['co2'] = {}
    projection['co2']['p1'], projection['co2']['p2'] = getPoints(r, _co2)
    projection['co2']['m'] = M(projection['co2']['p1'], projection['co2']['p2'])
    projection['co2']['b'] = B(projection['co2']['p1'], projection['co2']['m'])

    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    projection['nh4'] = {}
    projection['nh4']['p1'], projection['nh4']['p2'] = getPoints(r, _nh4)
    projection['nh4']['m'] = M(projection['nh4']['p1'], projection['nh4']['p2'])
    projection['nh4']['b'] = B(projection['nh4']['p1'], projection['nh4']['m'])


    projection['tolueno'] = {}
    projection['tolueno']['p1'], projection['tolueno']['p2'] = getPoints(r, _tolueno)
    projection['tolueno']['m'] = M(projection['tolueno']['p1'], projection['tolueno']['p2'])
    projection['tolueno']['b'] = B(projection['tolueno']['p1'], projection['tolueno']['m'])

    projection['acetona'] = {}
    projection['acetona']['p1'], projection['acetona']['p2'] = getPoints(r, _acetona)
    projection['acetona']['m'] = M(projection['acetona']['p1'], projection['acetona']['p2'])
    projection['acetona']['b'] = B(projection['acetona']['p1'], projection['acetona']['m'])
    
    if r > 0.05:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(vrl[idx], 3),
               round(r, 2),
               round(PPM(r, projection['co2']['m'],     projection['co2']['b']),2),
               round(PPM(r, projection['co']['m'],      projection['co']['b']),2),
               round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
               round(PPM(r, projection['nh4']['m'],     projection['nh4']['b']),2),
               round(PPM(r, projection['tolueno']['m'], projection['tolueno']['b']),2),
               round(PPM(r, projection['acetona']['m'], projection['acetona']['b']),2),
              ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(vrl[idx], 3),
               round(r, 2),
               round(PPM(r, projection['co2']['m'],     projection['co2']['b']),2),
               "----",
               round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
               round(PPM(r, projection['nh4']['m'],     projection['nh4']['b']),2),
               round(PPM(r, projection['tolueno']['m'], projection['tolueno']['b']),2),
               round(PPM(r, projection['acetona']['m'], projection['acetona']['b']),2),
              ))

    Volt     Ratio    CO2           CO            Alcohol       NH4           Tolueno       Acetona      
0   0.28     3.6      2.65          4.07          1.24          4.24          0.5           0.37         
1   0.38     2.6      6.96          13.92         3.58          9.5           1.59          1.17         
2   0.48     2.01     14.79         36.41         8.21          17.83         3.92          2.89         
3   0.58     1.63     27.61         80.77         16.33         30.04         8.26          6.13         
4   0.68     1.36     47.23         193.23        29.51         47.05         15.69         11.69        
5   0.78     1.16     75.86         449.8         49.76         69.9          27.65         20.69        
6   0.88     1.0      113.92        962.46        79.63         99.83         46.03         34.57        
7   0.98     0.88     159.75        1929.3        119.68        141.74        73.36         55.28        
8   1.08     0.78     218.43        3672.16   

In [23]:
## Projection - Increasing VRL

# Sensor 2

Vcc    = 5 #volts
vrl    = np.arange(sensor2['Vrl'], 4.5, 0.1)
RL     = 10 #kOhms 

projection = {}
projection['rs'] = [Rs(Vcc, v, RL) for v in vrl]
projection['ro'] = sensor2['ro']
projection['ratio'] = [Ratio(rs, projection['ro']) for rs in projection['rs']]

#print(projection['rs'])
#print(projection['ratio'])

print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format('','Volt', 'Ratio','CO2', 'CO', 'Alcohol', 'NH4', 'Tolueno', 'Acetona'))

for idx in range(len(projection['ratio'])):

    r = projection['ratio'][idx]
    projection['co2'] = {}
    projection['co2']['p1'], projection['co2']['p2'] = getPoints(r, _co2)
    projection['co2']['m'] = M(projection['co2']['p1'], projection['co2']['p2'])
    projection['co2']['b'] = B(projection['co2']['p1'], projection['co2']['m'])

    projection['co'] = {}
    projection['co']['p1'], projection['co']['p2'] = getPoints(r, _co)
    projection['co']['m'] = M(projection['co']['p1'], projection['co']['p2'])
    projection['co']['b'] = B(projection['co']['p1'], projection['co']['m'])

    projection['alcohol'] = {}
    projection['alcohol']['p1'], projection['alcohol']['p2'] = getPoints(r, _alcohol)
    projection['alcohol']['m'] = M(projection['alcohol']['p1'], projection['alcohol']['p2'])
    projection['alcohol']['b'] = B(projection['alcohol']['p1'], projection['alcohol']['m'])

    projection['nh4'] = {}
    projection['nh4']['p1'], projection['nh4']['p2'] = getPoints(r, _nh4)
    projection['nh4']['m'] = M(projection['nh4']['p1'], projection['nh4']['p2'])
    projection['nh4']['b'] = B(projection['nh4']['p1'], projection['nh4']['m'])


    projection['tolueno'] = {}
    projection['tolueno']['p1'], projection['tolueno']['p2'] = getPoints(r, _tolueno)
    projection['tolueno']['m'] = M(projection['tolueno']['p1'], projection['tolueno']['p2'])
    projection['tolueno']['b'] = B(projection['tolueno']['p1'], projection['tolueno']['m'])

    projection['acetona'] = {}
    projection['acetona']['p1'], projection['acetona']['p2'] = getPoints(r, _acetona)
    projection['acetona']['m'] = M(projection['acetona']['p1'], projection['acetona']['p2'])
    projection['acetona']['b'] = B(projection['acetona']['p1'], projection['acetona']['m'])
    
    if r > 0.05:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(vrl[idx], 3),
               round(r, 2),
               round(PPM(r, projection['co2']['m'],     projection['co2']['b']),2),
               round(PPM(r, projection['co']['m'],      projection['co']['b']),2),
               round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
               round(PPM(r, projection['nh4']['m'],     projection['nh4']['b']),2),
               round(PPM(r, projection['tolueno']['m'], projection['tolueno']['b']),2),
               round(PPM(r, projection['acetona']['m'], projection['acetona']['b']),2),
              ))
    else:
        print ("{:<3} {:<8} {:<8} {:<13} {:<13} {:<13} {:<13} {:<13} {:<13}"
       .format(idx,
               round(vrl[idx], 3),
               round(r, 2),
               round(PPM(r, projection['co2']['m'],     projection['co2']['b']),2),
               "----",
               round(PPM(r, projection['alcohol']['m'], projection['alcohol']['b']),2),
               round(PPM(r, projection['nh4']['m'],     projection['nh4']['b']),2),
               round(PPM(r, projection['tolueno']['m'], projection['tolueno']['b']),2),
               round(PPM(r, projection['acetona']['m'], projection['acetona']['b']),2),
              ))

    Volt     Ratio    CO2           CO            Alcohol       NH4           Tolueno       Acetona      
0   0.48     3.6      2.65          4.07          1.24          4.24          0.5           0.37         
1   0.58     2.92     4.95          9.02          2.46          7.15          1.06          0.77         
2   0.68     2.43     8.47          17.89         4.44          11.2          2.01          1.48         
3   0.78     2.07     13.61         32.76         7.49          16.64         3.55          2.62         
4   0.88     1.79     20.85         56.46         11.99         23.76         5.91          4.37         
5   0.98     1.57     30.8          92.87         18.43         32.92         9.42          6.99         
6   1.08     1.39     44.19         171.66        27.43         44.51         14.5          10.79        
7   1.18     1.24     61.93         313.33        39.79         59.0          21.7          16.21        
8   1.28     1.11     85.14         552.63    